In [ ]:
from __future__ import print_function, division

In [ ]:
import requests
from pprint import pprint

## Weather API

* api.openweathermap.org

### Getting and saving API Keys

In [ ]:
# Get your API Key from api.openweathermap.org
# NEVER EVER Store API Key in Jupyter Notebooks and check them in
API_KEY = ""  

In [ ]:
# Store your config in separate python file or JSON or YAML and load the config.
# import json

# json.load("your config keys.file")
# from config import API_KEY

### Querying Weather API

In [ ]:
# api.openweathermap.org/data/2.5/weather?zip={zip code},{country code}

base_url = "http://api.openweathermap.org/data/2.5/weather"

results = []
for city in ["San Francisco", "Austin", "New York", "Seattle", "London,GB", "Berlin,DE"]:
    params = {
        "q": city,
    #     "zip": "94550,us",
        "units": "imperial",  # for metric units - "metric"
        "appid": API_KEY  # Add your API Key from api.openweathermap.org
    }

    r = requests.get(base_url, params=params)
    print(r.json())
    results.append(r.json())

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
df = json_normalize(results)

In [ ]:
df.head()

## Extracting Weather

In [ ]:
df.weather[0]

In [ ]:
pd.Series(df.weather)

In [ ]:
def get_main_values(rows):
    return pd.Series(rows[0])
df_weather = df.weather.apply(get_main_values)

In [ ]:
df_weather.columns = ["weather_" + column for column in df_weather.columns]

In [ ]:
# Now df_weather has all the Weather information
df_weather

## Merge this with the main Dataframe

In [ ]:
df.drop('weather', axis=1, inplace=True)

In [ ]:
df = pd.merge(df, df_weather, left_index=True, right_index=True)

In [ ]:
df

### Achievent Unlocked.  We have now merged Weather with the rest of the fields

## Now let's get 16 day forcast

In [ ]:
# http://api.openweathermap.org/data/2.5/forecast/daily
# ?q=Berlin&mode=xml&units=metric&cnt=7&appid=39cbe69c64ec3521b031c036702a5ef1

base_url = "http://api.openweathermap.org/data/2.5/forecast/daily"

results = []
for city in ["San Francisco", "Austin", "New York", "Seattle"]:
    params = {
        "q": city,
    #     "zip": "94550,us",
        "units": "imperial",
        "mode": "json",
        "cnt": 16,
        "appid": API_KEY  # Add your API Key from api.openweathermap.org
    }

    r = requests.get(base_url, params=params)
    results.append(r.json())

In [ ]:
results[0]

In [ ]:
df = json_normalize(results)

In [ ]:
df

## Now it's much harder to parse out the 16 days of weather and flatten in a single DF

* In MongoDB or other NoSQL DBs we could just store the whole docuemnt as-is and extract what we need